# Evalutation of the v6_base_with_einzel model

extract the emittances for all offsets and safe in a csv

In [14]:
import os
import numpy as np
import pandas as pd
import altair as alt
alt.enable_mime_rendering()
import scipy.constants

import PhaseSpaceEval.monitor_quantities as monq
from PhaseSpaceEval.import_particle_data import *
from PhaseSpaceEval.trajectory import Trajectory
from PhaseSpaceEval.particlemonitor import ParticleMonitor

## Import Raw Data and set filename *!!! ADJUST FILENAMES HERE !!!*

In [15]:
MODELNAME = "v6_base_with_einzel"
RAW_PATH = "rawdata_" + MODELNAME + "/"
EMIT_FILENAME = "emit_" + MODELNAME + ".csv" # Name for the emittance output file
MON_FILENAME = "mon_" + MODELNAME + ".csv" # Name for the monitors output file

particle_source_names = import_source_names(RAW_PATH + MODELNAME + "-source_names.txt")
particle_constants = import_particle_constants(RAW_PATH + MODELNAME + "-constants.txt")
particle_trajectories = import_particle_trajectories(RAW_PATH + MODELNAME + "-trajectories.txt")

## Create convenient lists
### Raw Data

In [16]:
# Delete the single_centre source, not required
for key in particle_source_names.keys():
    if particle_source_names[key] == "single_centre":
        del(particle_source_names[key])
        break
        
# Generate simple list with all source IDs
sourceIDs = list(particle_source_names.keys())
#print(sourceIDs)

# Create lists with all particles belonging to a source and with the id of the central particles
particlesBySrc = dict() # dict for all particleIDs
centresBySrc = dict() # dict with the ids of the central particles
for sID in sourceIDs:
    pIDs = particle_constants["particleID"].loc[particle_constants["sourceID"] == sID].tolist()
    particlesBySrc.update({sID : pIDs})
    centresBySrc.update({sID : min(pIDs)}) # the smallest pID for each source is the centre
#print(particlesBySrc)
#print(centresBySrc)

### Trajectories

In [17]:
# Create Trajectories
trajsBySrc = dict() # Dict for all trajectories
ctrajsBySrc = dict() # Dict for central trajectories
lostParticles = list()
for sID in sourceIDs:
    # Compute central trajectory for each sID
    cID = centresBySrc[sID]
    ctr = Trajectory(particle_trajectories[cID],
                     particle_constants.loc[particle_constants["particleID"] == cID].squeeze())
    ctrajsBySrc.update({sID : ctr})

    # For each sID compute the trajectories of all pIDs
    # Note Particles that cannot be found in trajectory dataframe, these were lost
    pIDs = particlesBySrc[sID]
    trajs = list()
    for pID in pIDs:
        try:
            tr = Trajectory(particle_trajectories[pID],
                            particle_constants.loc[particle_constants["particleID"] == pID].squeeze())
        except KeyError:
            lostParticles.append(pID)
        trajs.append(tr)
    trajsBySrc.update({sID : trajs})
print(lostParticles)

[11838]


### Monitors

In [18]:
# Create Monitors
monBySrc = dict()
for sID in sourceIDs:
    ctr = ctrajsBySrc[sID]
    t0 = ctr.find_time("z", 770)
    mon = ParticleMonitor(time0=t0, trajectory=ctr)
    monBySrc.update({sID : mon})

## Record Monitor Interactions

In [19]:
# Record Monitor Interactions
for sID in sourceIDs:
    mon = monBySrc[sID]
    mon.reset_events()
    mon.reset_misses()
    for tr in trajsBySrc[sID]:
        mon.record_intersect(tr)

# Read Out misses and events
missesBySrc = dict()
eventsBySrc = dict()
for sID in sourceIDs:
    missesBySrc.update({sID : monBySrc[sID].get_misses()})
    eventsBySrc.update({sID : monBySrc[sID].get_events()})
#print(missesBySrc)

# Add lost particles to miss counter
for pID in lostParticles:
    for sID in sourceIDs:
        if pID in particlesBySrc[sID]:
            missesBySrc[sID] += 1
#print(missesBySrc)

## Export Monitor Interactions

In [20]:
export_events = pd.DataFrame()
for sID in sourceIDs:
    events = eventsBySrc[sID].copy()
    events["sourceID"] = sID
    events["sourceName"] = particle_source_names[sID]
    export_events = export_events.append(events, ignore_index=True)
export_events.to_csv(MON_FILENAME)

## Compute and save emittances

In [21]:
# Compute and save emittances
colnames = ["sourceID", "sourceName", "x_offset", "y_offset", "x_emittance", "y_emittance",
            "x_norm_emittance", "y_norm_emittance", "losses", "relbeta"]
emit_df = pd.DataFrame(columns=colnames)
emit_temp = pd.DataFrame([np.zeros(len(colnames))], columns=colnames)
for sID in sourceIDs:
    name = particle_source_names[sID]
    xoff = float(name.split('_')[1])
    yoff = float(name.split('_')[3])
    xemit = monq.emittance_u(eventsBySrc[sID])
    yemit = monq.emittance_v(eventsBySrc[sID])
    # Compute rel. beta for the screen 1e6 mm/ns ->m/s
    beta = monBySrc[sID].abs_vel / scipy.constants.speed_of_light * 1e6
    xemit_n = beta * (1 - beta**2)**(-0.5) * xemit # beta * gamme * emit
    yemit_n = beta * (1 - beta**2)**(-0.5) * yemit
    losses = missesBySrc[sID]
    
    emit_temp["sourceID"] = sID
    emit_temp["sourceName"] = name
    emit_temp["x_offset"] = xoff
    emit_temp["y_offset"] = yoff
    emit_temp["x_emittance"] = xemit
    emit_temp["y_emittance"] = yemit
    emit_temp["x_norm_emittance"] = xemit_n
    emit_temp["y_norm_emittance"] = yemit_n
    emit_temp["losses"] = losses
    emit_temp["relbeta"] = beta

    emit_df = emit_df.append(emit_temp, ignore_index=True)

emit_df.sort_values(["x_offset", "y_offset"], inplace=True)
emit_df.reset_index(inplace=True, drop=True)

# Load emittance data from startinplane to compute relative emittance growth
emit_start = pd.read_csv("emit_startingplane.csv", index_col=0)
emit_df["x_emittance_rel"] = emit_df["x_emittance"]/emit_start["x_emittance"]
emit_df["y_emittance_rel"] = emit_df["y_emittance"]/emit_start["y_emittance"]
# Save emittance table
emit_df.to_csv(EMIT_FILENAME)
# Show head for debugging purposes
emit_df

,sourceID,sourceName,x_offset,y_offset,x_emittance,y_emittance,x_norm_emittance,y_norm_emittance,losses,relbeta,x_emittance_rel,y_emittance_rel
0,16777222,x_-15_y_0,-15.0,0.0,1.654207,1.571951,0.003819,0.003629,0,0.002308,1.020401,0.999806
1,16777229,x_-15_y_5,-15.0,5.0,1.622523,1.590093,0.003746,0.003671,0,0.002308,1.018487,0.997749
2,16777236,x_-15_y_10,-15.0,10.0,1.648748,1.596193,0.003806,0.003685,0,0.002308,1.019564,0.999692
3,16777243,x_-15_y_15,-15.0,15.0,1.719856,1.660899,0.003970,0.003834,0,0.002308,1.029125,0.999411
4,16777220,x_-10_y_0,-10.0,0.0,1.593118,1.556111,0.003678,0.003592,0,0.002308,1.000622,1.002257
5,16777227,x_-10_y_5,-10.0,5.0,1.606309,1.583458,0.003708,0.003655,0,0.002308,0.995861,1.002158
6,16777234,x_-10_y_10,-10.0,10.0,1.497294,1.679003,0.003456,0.003876,0,0.002308,1.007150,1.008297
7,16777241,x_-10_y_15,-10.0,15.0,1.466815,1.644204,0.003386,0.003796,0,0.002308,1.011706,1.006070
8,16777218,x_-5_y_0,-5.0,0.0,1.680502,1.721558,0.003879,0.003974,0,0.002308,1.002473,1.002317
9,16777225,x_-5_y_5,-5.0,5.0,1.536611,1.641997,0.003547,0.003790,0,0.002308,1.008656,1.010032


## Plots

In [22]:
alt.Chart(emit_df).mark_line().encode(
    color='y_offset:N',
    x='x_offset',
    y='x_emittance',
)

<altair.VegaLite object>

In [23]:
alt.Chart(emit_df).mark_line().encode(
    color='y_offset:N',
    x='x_offset',
    y='y_emittance',
)

<altair.VegaLite object>

In [24]:
alt.Chart(emit_df).mark_line().encode(
    color='y_offset:N',
    x='x_offset',
    y='x_emittance_rel',
)

<altair.VegaLite object>

In [25]:
alt.Chart(emit_df).mark_line().encode(
    color='y_offset:N',
    x='x_offset',
    y='y_emittance_rel',
)

<altair.VegaLite object>

In [26]:
alt.Chart(emit_df).mark_line().encode(
    color='y_offset:N',
    x='x_offset',
    y='losses',
)

<altair.VegaLite object>